<a href="https://colab.research.google.com/github/choarauc/form/blob/main/BeautifoulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(a) Exécuter les précédentes lignes de code et afficher le contenu de la liste titre_SC.

(b) créer les listes annee_sortie_SC et note_SC contenant respectivement l'année de sortie et la note de chaque film présent dans la liste.

(c) Utiliser la méthode strip() pour "nettoyer" les listes obtenues précédemment (retirer les parenthèses : "()" pour l'année de sortie, les espaces inutiles pour la note).
 

> Bloc en retrait   La méthode strip("char") permet de supprimer "char" d'une liste ou chaîne de caractères donnée. Quand aucun argument n'est précisé cela retire tous les espaces de la liste ou chaîne de caractères en question.


(d) Créer un DataFrame nommé sens_critique à partir de ces trois listes. Les colonnes de ce DataFrame porteront les noms Titre, Annee_sortie_SC et Note_SC.> Bloc en retrait





In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

page_SC = urlopen("https://www.senscritique.com/films/tops/top111")

soup = BeautifulSoup(page_SC, 'html.parser')

noms_SC = soup.findAll(name='a', attrs={'class': 'elco-anchor'})
titre_SC = [] # On crée une liste vide qui contiendra tous les titres propres
for element in noms_SC:
    titre_SC.append(element.text)
    
print(titre_SC)

['Douze Hommes en colère', 'Harakiri', 'Le Bon, la Brute et le Truand', 'Barberousse', 'Les Sept Samouraïs', "Il était une fois dans l'Ouest", 'Il était une fois en Amérique', 'Le Parrain', 'Le Trou', 'Point limite', "Entre le ciel et l'enfer", 'Le Voyage de Chihiro', 'Princesse Mononoké', "L'Aurore", 'Dersou Ouzala', 'Sherlock Junior', "Vol au-dessus d'un nid de coucou", 'Pour Sama', 'Parasite', 'Le Parrain - 2e Partie', 'Pulp Fiction', 'Apocalypse Now', 'Le Dictateur', "Voyage au bout de l'enfer", 'Psychose', 'La Femme des sables', 'Soy Cuba', 'Requiem pour un massacre', 'Sátántangó – Le Tango de Satan', 'Les Affranchis', 'Boulevard du crépuscule', 'Quand passent les cigognes', 'Rébellion', 'Le Tombeau des lucioles', 'Les Enfants du paradis', 'Ève...', 'Andreï Roublev', 'Rendez-vous', 'Les Temps modernes', 'La vie est belle', 'Les Lumières de la ville', 'Jeux dangereux', 'La Foule', 'M le maudit', 'Les Sentiers de la gloire', 'Le Limier', 'Fanny et Alexandre', 'Metropolis', 'Stalker'

In [2]:
annee_sortie_SC = []
for element in soup.findAll('span', attrs={'class': 'elco-date'}):
    annee_sortie_SC.append(element.text.strip("()")) # On retire les parenthèses

note_SC = []
for element in soup.findAll('a', attrs={'class': 'erra-global'}):
    note_SC.append(element.text.strip()) # On retire les espaces inutiles
    
# Création du DataFrame

sens_critique = pd.DataFrame(list(zip(titre_SC,annee_sortie_SC,note_SC)), columns=["Titre","Annee_sortie_SC","Note_SC"])

sens_critique.head()

,Titre,Annee_sortie_SC,Note_SC
0,Douze Hommes en colère,1957,8.7
1,Harakiri,1962,8.6
2,"Le Bon, la Brute et le Truand",1966,8.5
3,Barberousse,1965,8.5
4,Les Sept Samouraïs,1954,8.5


In [3]:
page_imdb = urlopen("https://www.imdb.com/chart/top")

soup = BeautifulSoup(page_imdb, 'html.parser')

titre_imdb = []
for element in soup.select(".titleColumn a"):
    titre_imdb.append(element.text)
    
annee_sortie_imdb = []
for element in soup.findAll(name='span', attrs={'class': 'secondaryInfo'}):
    annee_sortie_imdb.append(element.text.strip("()"))
    
note_imdb = []
for element in soup.select('.ratingColumn strong'):
    note_imdb.append(element.text)

# Création du DataFrame

imdb = pd.DataFrame(list(zip(titre_imdb,annee_sortie_imdb,note_imdb)), columns=["Titre","Annee_sortie_imdb","Note_imdb"])

imdb.head()



,Titre,Annee_sortie_imdb,Note_imdb
0,The Shawshank Redemption,1994,9.2
1,The Godfather,1972,9.2
2,The Dark Knight,2008,9.0
3,The Godfather: Part II,1974,9.0
4,12 Angry Men,1957,9.0


(f) Utiliser la méthode str.upper() pour mettre en majuscule l'intégralité de la colonne Titre.

(g) Réaliser le merge des deux DataFrames sur la colonne Titre en ne sélectionnant que l'intersection des deux.

(h) Transformer le type des colonnes Note_IMDB et Note_SC en format numérique si cela n'est pas déjà fait.

(i) Faire un test de Student apparié pour comparer les moyennes données par Sens Critique et par IMDB à l'aide de la fonction ttest_rel() du module scipy.stats.

In [4]:
from scipy.stats import ttest_rel

# Retirer les accents de la colonne "Titre"

imdb["Titre"] = imdb["Titre"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
sens_critique["Titre"] = sens_critique["Titre"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Mettre tout en majuscule

imdb["Titre"] = imdb["Titre"].str.upper()
sens_critique["Titre"] = sens_critique["Titre"].str.upper()

# Merge

note_finale = imdb.merge(sens_critique, how="inner", left_on="Titre", right_on="Titre", )

note_finale["Note_imdb"] = pd.to_numeric(note_finale["Note_imdb"])
note_finale["Note_SC"] = pd.to_numeric(note_finale["Note_SC"])

# T-test

ttest_rel(note_finale["Note_imdb"],note_finale["Note_SC"])

# La p-value du test de Student est inférieure à 0,05, on rejette donc l'hypothèse que les 
# moyennes sont égales. Comme la statistique de test est positive, la différence entre la moyenne 
# IMDB et la moyenne Sens Critique est elle aussi positive, ce qui suggère que les utilisateurs 
# IMDB sont plus cléments que les utilisateurs Sens Critique quant à la notation.

Ttest_relResult(statistic=3.041239048167191, pvalue=0.011221731797856767)

# WEBSCRAPPING GOOGLE

In [5]:
from googlesearch import search

for url in search('parité en intelligence artificielle', tld='com', lang='fr', start=0, stop=10):
    print(url)



https://datascientest.com/parite-en-intelligence-artificielle-un-enjeu-cle
https://www.laboratoiredelegalite.org/le-pacte-pour-une-intelligence-artificielle-egalitaire-entre-les-femmes-et-les-hommes/
https://theconversation.com/pourquoi-est-il-important-davoir-une-egalite-femmes-hommes-dans-le-monde-de-lia-157619
https://www.actuia.com/tag/parite/
https://www.echosciences-grenoble.fr/communautes/marie-rosalind-jane-et-les-sciences/articles/women-in-ai-vers-la-parite-dans-le-monde-du-numerique-et-de-l-intelligence-artificielle
https://www.lesechos.fr/idees-debats/cercle/opinion-pas-de-diversite-des-genres-sans-intelligence-artificielle-256956
https://www.thalesgroup.com/fr/monde/groupe/news/technologies-et-parite-comment-faire-une-ia-non-discriminatoire
https://www.europe1.fr/technologies/edito-lintelligence-artificielle-est-elle-sexiste-3932234
https://rm.coe.int/pacte-ia-laboratoire-egalite-fr/1680a05783
https://apolitical.co/solution-articles/fr/les-femmes-craignent-l-intelligence-ar

Web Scraping avec le package NewsPaper

In [ ]:
pip install newspaper

In [ ]:
from newspaper import Article

url = 'https://datascientest.com/parite-en-intelligence-artificielle-un-enjeu-cle'
article = Article(url)
article.download()
article.parse()
print(article.text)

 Récupérer les auteurs, la date de publication, le résumé et les mots clefs de l'article de notre blog sur la parité en intelligence artificielle.

In [ ]:
article.nlp() # on fait avant tout fonctionner le langage naturel
article.authors 
article.publish_date
article.summary
article.keywords
